In [80]:
import nltk
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

In [2]:
!ls

airbnb_listings_descriptions.xlsx
airbnb_reviews_EDA_cleaning.ipynb
airbnb_text_reviews.xlsx
listings.csv
reviews.csv


In [3]:
reviews = pd.read_excel('airbnb_text_reviews.xlsx')

In [4]:
reviews.sample(1)

,listing_id,id,date,reviewer_id,reviewer_name,comments
336934,10253315,89644525,2016-07-29,30964687,Mulham,Last and her husband are wonderful people. The...


In [5]:
reviews.sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
613439,24288869,485391963326243968,2021-10-31,426982029,Conner,"Check in 2+ hours late, finally allowed in and..."
246177,6326681,621006676601798016,2022-05-06,46496730,Jackie,I stayed here for a month and loved my stay! T...
200086,4339924,21772861,2014-10-23,22762693,Grant,"Great place to stay! The apartment is large, c..."


In [8]:
reviews['comments'] = reviews['comments'].astype('string')

In [9]:
reviews['comments'].describe()

count         1048358
unique        1000410
top       Great place
freq             1366
Name: comments, dtype: object

## removing rows with only punctuation

In [17]:
import re, string

pattern = re.compile("[\d{}]+$".format(re.escape(string.punctuation)))

x = ['12,523', '3.46', "this is not", "foo bar 42", "23fa"]

[item for item in x if pattern.match(item)]

['12,523', '3.46']

In [18]:
reviews.loc[(reviews['comments'] == '.') | (reviews['comments'] == '!') | (reviews['comments'] == '?') ]

,listing_id,id,date,reviewer_id,reviewer_name,comments
609,5178,458595941119654016,2021-09-24,409836786,Elya,.
8075,27644,503596934129974976,2021-11-25,38781491,Jay,.
9103,31994,180524495,2017-08-10,98704132,Martin (Aka Max),.
9106,31994,194910000,2017-09-17,99756545,Vardhan,.
9159,31994,366953207,2019-01-03,213464080,Glenda,.
...,...,...,...,...,...,...
1045288,593313785791808000,645676525698039040,2022-06-09,461019684,Devon,.
1046154,596511810816093056,618129884161660032,2022-05-02,455819359,Tatiana,.
1046404,597180195652908032,610077003524514944,2022-04-21,421646015,Guillermo,.
1047280,599993984917025024,623911135936929024,2022-05-10,46317236,Alexandre,.


In [19]:
reviews['comments'].dropna(inplace = True)

In [191]:
reviews = reviews.head(100)

In [213]:
review_list = reviews['comments'].to_list()
review_list

['Great host ',
 'Nice room for the price. Great neighborhood. John was very accommodating. Bottles of water in the room were a nice touch and very much appreciated.',
 'Very nice apt.  New remodeled.',
 "Great place to stay for a while. John is a great host and a great man. The house is very homy. You'll feel right at home. ",
 '   .',
 "I really enjoyed my time here in deep south Brooklyn and John was an excellent host - very attentive to any concerns that I had and a very welcome presence when he's around. <br/><br/>The room itself is a great space with a super-comfy bed and lovely traditional ceiling fan. The street is pleasant, tree-lined and quiet and the townhouse is located a short, convenient walk from the F train, which has been perfect for getting to my office here.<br/><br/>It's easy to see why John's a Super Host and I look forward to staying with him again!",
 "John was a great host. He was very responsive before and during our visit. He really wanted to make sure we had 

In [214]:
review_string = '@'.join(review_list)
review_string

'Great host @Nice room for the price. Great neighborhood. John was very accommodating. Bottles of water in the room were a nice touch and very much appreciated.@Very nice apt.  New remodeled.@Great place to stay for a while. John is a great host and a great man. The house is very homy. You\'ll feel right at home. @   .@I really enjoyed my time here in deep south Brooklyn and John was an excellent host - very attentive to any concerns that I had and a very welcome presence when he\'s around. <br/><br/>The room itself is a great space with a super-comfy bed and lovely traditional ceiling fan. The street is pleasant, tree-lined and quiet and the townhouse is located a short, convenient walk from the F train, which has been perfect for getting to my office here.<br/><br/>It\'s easy to see why John\'s a Super Host and I look forward to staying with him again!@John was a great host. He was very responsive before and during our visit. He really wanted to make sure we had a good stay. It was o

In [215]:
# Initialize spacy 'en_core_web_sm' model, keeping only tagger component needed for lemmatization

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Parse the sentence using the loaded 'en_core_web_sm' model object `nlp`
doc = nlp(review_string)

# Extract the lemma for each token and join
lemmatized_corpus = " ".join([token.lemma_.lower() for token in doc if not (token.is_punct)])

lemmatized_corpus_list = lemmatized_corpus.split(sep = "@")

In [216]:
lemmatized_corpus_list

['great host ',
 'nice room for the price great neighborhood john be very accommodating bottle of water in the room be a nice touch and very much appreciated.',
 'very nice apt   new remodeled.',
 "great place to stay for a while john be a great host and a great man the house be very homy you 'll feel right at home    .",
 "i really enjoy my time here in deep south brooklyn and john be an excellent host very attentive to any concern that i have and a very welcome presence when he be around < br/><br/>the room itself be a great space with a super comfy bed and lovely traditional ceiling fan the street be pleasant tree line and quiet and the townhouse be locate a short convenient walk from the f train which have be perfect for get to my office here.<br/><br/>it 's easy to see why john 's a super host and i look forward to stay with he again!",
 'john be a great host he be very responsive before and during our visit he really want to make sure we have a good stay it be our 1st airbnb expe

## Time to clean our text

In [212]:
reviews['cleaned_tokens'] = lemmatized_corpus_list

ValueError: Length of values (99) does not match length of index (100)